## Skew-symmetric Matrix Completion

In [65]:
import numpy as np
import scipy.stats
import scipy.stats
import matplotlib.pyplot as plt

### Algorithm 1: Gradient Descent

Let $M$ dentoe the observed matrix, $\Omega$ denote the indicator matrix (0:observed;1:observed)

Any rank-r (note r must be an even numeber) skew-symmetric matrix $M$ admits a linear factorization $M = AB'-BA'$, where $A,B \in R^{n\times {r\over2}}$.

This algorithm minimizes $\frac{1}{2p}||P_\Omega(M-(AB'-B'A))||^2_F + {1\over 4}||A'A-B'B||^2_F + {1\over 4}||A'B+B'A||^2_F$ via gradient descent. 

Initialization is based on schur decoposition. Let $(A_0,B_0) = Schur({1\over p}M)$, where $p$ is the empirical sample rate.

In [228]:
def MC_GD(M,Omega,r,maxIter = 1000,tol = 1e-3):
    """
    References
    ----------
    .. [1] Chen, Ji, Xiaodong Li, and Zongming Ma. "Nonconvex matrix completion with linearly parameterized factors." Journal of Machine Learning Research 23.207 (2022): 1-35.
    .. [2] Sun, Ruoyu, and Zhi-Quan Luo. "Guaranteed matrix completion via non-convex factorization." IEEE Transactions on Information Theory 62.11 (2016): 6535-6579.
    .. [3] Chi, Yuejie. "Low-rank matrix completion [lecture notes]." IEEE Signal Processing Magazine 35.5 (2018): 178-181.
    
    Parameters:
    ----------   
    M: array
        Observed matrix.
    Omega: array
        Indicator matrix (0:observed;1:observed).
    r: even integer
        Desired rank.
    maxIter: integer, optional
        Maximum allowed iteration. Default is 1000.
    tol: float, optional
        Absolute tolerances. Default is 1e-3.
    
    Returns:
    ----------   
    M_rec: array
        Recovered matrix.
    
    """         
    if(r%2 != 0):
        raise ValueError("r must be an even number")
        
    N = len(M);
    
    # objective function
    obj = lambda A,B,M,p: 1/(2*p)*np.linalg.norm(Omega*(A@B.T-B@A.T-M),'fro')**2+1/4*np.linalg.norm(A.T@A-B.T@B,'fro')**2+1/4*np.linalg.norm(A.T@B+B.T@A,'fro')**2
    
    # empirical sample rate
    p = sum(sum(Omega))/N**2;
    
    # Initialization
    U,Q = scipy.linalg.schur((1/p)*M)
    A = np.zeros((N,int(r/2)))
    B = np.zeros((N,int(r/2)))
    for i in range(int(r/2)):
        if (U[2*i+1,2*i]>0):
            A[:,i] = np.sqrt(U[2*i+1,2*i])*Q[:,2*i+1]
            B[:,i] = np.sqrt(U[2*i+1,2*i])*Q[:,2*i]
        elif (U[2*i+1,2*i]<0):
            A[:,i] = np.sqrt(-U[2*i+1,2*i])*Q[:,2*i]
            B[:,i] = np.sqrt(-U[2*i+1,2*i])*Q[:,2*i+1]
    # gradient
    for i in range(maxIter):
        # gradient
        gradA = 1/p*(Omega*(A@B.T-B@A.T-M)@B-Omega*(B@A.T-A@B.T+(-M).T)@B)+A@(A.T@A-B.T@B)+B@(B.T@A+A.T@B)
        gradB = 1/p*(Omega*(B@A.T-A@B.T+(-M).T)@A-Omega*(A@B.T-B@A.T-M)@A)-B@(A.T@A-B.T@B)+A@(B.T@A+A.T@B)
        # print(np.linalg.norm(gradB,'fro'))
        # step size    
        k = 1
        while (obj(A-10**k*gradA,B-10**k*gradB,M,p)>obj(A,B,M,p)) & (k>=-20):
            k = k-1
            
        # update
        A = A - 10**k*gradA
        B = B - 10**k*gradB
        
        # print information
        if (i == 0) | (i%50 == 0) | (np.linalg.norm(np.concatenate((gradA,gradB)),'fro') < tol):
            print("iter: %d, obj: %f \n"%(i,obj(A,B,M,p)))
            
        if (np.linalg.norm(np.concatenate((gradA,gradB)),'fro') < tol):
            break
    return A@B.T-B@A.T

    

### Example

In [70]:
pokem = np.load("Data and Examples\Data and Examples\Pokemon\F.npy")

In [238]:
FM = pokem;
N = len(FM);

# Random observation matrix
samp_rate = 0.5;  # sampling rate
Omega = scipy.stats.bernoulli.rvs(size=N*N,p=samp_rate).reshape(N,N)

for i in range(1,N):
    for j in range(N-1):
        Omega[j,i] = Omega[i,j];

for i in range(N):
    Omega[i,i] = 1;

# observed matrix
OM = FM*Omega

In [229]:
M_rec = MC_GD(OM,Omega,16,maxIter = 1000,tol = 1e-3)

iter: 0, obj: 139257.280283 

iter: 50, obj: 13276.149406 

iter: 100, obj: 9036.760947 

iter: 150, obj: 6819.612569 

iter: 200, obj: 5415.770732 

iter: 250, obj: 4776.876209 

iter: 300, obj: 4473.514263 

iter: 350, obj: 4326.169054 

iter: 400, obj: 4265.747965 

iter: 450, obj: 4241.183495 

iter: 500, obj: 4229.447780 

iter: 550, obj: 4222.581896 

iter: 600, obj: 4217.920836 

iter: 650, obj: 4214.466524 

iter: 700, obj: 4211.838288 

iter: 750, obj: 4209.751027 

iter: 800, obj: 4208.040959 

iter: 850, obj: 4206.660305 

iter: 900, obj: 4205.517141 

iter: 950, obj: 4204.560401 



### Algorithm 2: Singular value thresholding

This algotirhm is based on SVD.

$\operatorname{minimize} \quad \lambda\|\boldsymbol{X}\|_*+\frac{1}{2}\left\|\mathcal{P}_{\Omega}(\boldsymbol{X})-\mathcal{P}_{\Omega}(\boldsymbol{M})\right\|_F^2$

Iterates
$\left\{\begin{array}{l}\boldsymbol{X}^k=\mathcal{D}_{\lambda \delta_{k-1}}\left(\boldsymbol{Y}^{k-1}\right) \\ \boldsymbol{Y}^k=\boldsymbol{X}^k+\delta_k P_{\Omega}\left(\boldsymbol{M}-\boldsymbol{X}^k\right)\end{array}\right.$,

where the soft-thresholding operator is defined as $\left.\mathcal{D}_\tau(\boldsymbol{X}):=\boldsymbol{U D}_\tau(\boldsymbol{\Sigma}) \boldsymbol{V}^*,\mathcal{D}_\tau(\boldsymbol{\Sigma})=\operatorname{diag}\left(\left\{\sigma_i-\tau\right)_{+}\right\}\right)$,

In [271]:
def MC_SVT(M,Omega,t=None,delta_t=2,maxIter=1000,tol=1e-2):
    """
    References
    ----------
    .. [1] Cai, Jian-Feng, Emmanuel J. Candès, and Zuowei Shen. "A singular value thresholding algorithm for matrix completion." SIAM Journal on optimization 20.4 (2010): 1956-1982.
    
    Parameters:
    ----------   
    M: array
        Observed matrix.
    Omega: array
        Indicator matrix (0:observed;1:observed).
    t: float,optimal
        Singular value threshood. Default is 2*N.
           
    delta_t: float, optional
        Step size. Default is 2. Try larger values if it's too slow. Try smaller values if it doesn't converge.
    maxIter: integer, optional
        Maximum allowed iteration. Default is 1000.
    tol: float, optional
        Absolute tolerances. Default is 1e-2.
    
    Returns:
    ----------   
    M_rec: array
        Recovered matrix.
    
    """  
    
    N = len(M)
    if t == None:
        t = 2*N
        
    X = Omega*M
    Y = np.zeros((N,N))
    iters = 0
    epsilon= 1e+5
    
    while (epsilon>tol) & (iters<maxIter):
        U,S,VT = np.linalg.svd(Y)
        S = S - t
        ind = S > 0
        S = S*ind 
        X = U@np.diag(S)@VT;  
        Y = Y + delta_t*Omega*(M-X)
        epsilon= np.linalg.norm(Omega*(M-X),'fro')/np.linalg.norm(Omega*M,'fro')
        iters = iters + 1
        if (iters == 1) | (iters%50 == 0) | (epsilon < tol):
            print("iter: %d, err: %f \n"%(iters,epsilon))
    return X
    

### Example

In [272]:
M_rec = MC_SVT(OM,Omega,delta_t = 2,maxIter = 1000, tol = 1e-2)

iter: 1, err: 1.000000 

iter: 50, err: 0.054070 

iter: 100, err: 0.037609 

iter: 150, err: 0.028676 

iter: 200, err: 0.023586 

iter: 250, err: 0.019411 

iter: 300, err: 0.016431 

iter: 350, err: 0.014113 

iter: 400, err: 0.012105 

iter: 450, err: 0.010615 

iter: 475, err: 0.009996 

